In [1]:
import datetime,time
from datetime import datetime
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import skmob
import gc
import sys
import math
from shapely.geometry import Point
from skmob.preprocessing import filtering
from skmob.preprocessing import detection
from geopy.distance import geodesic
from sklearn.neighbors import BallTree
from scipy.signal import savgol_filter

In [2]:
filename = 'Data/Proximity+200_Unlabeled_60_final.pickle'
with open(filename, 'rb') as f:
    Bus_All_Segment,SegmentID ,Rail_All_Segment, Traffic_All_Segment, Stage, Data_All_Segment, SegmentNumber = pickle.load(f, encoding='latin1')

# Calculating Motion channels

In [3]:
# Total_Segment_InSequence checks the number of GPS points for each instance in all Stages
Total_Segment_InSequence = []
# Save the 4 channels for each user separately
Total_RelativeDistance = []
Total_Speed = []
Total_Acceleration = []
Total_Jerk = []
Total_BearingRate = []
Total_Stage = []
Total_SegmentNumber = []
Total_Outlier = []
Total_Descriptive_Stat = []
Total_Delta_Time = []
Total_Velocity_Change = []
Total_BusLine = []
Total_Railway = []
Total_Traffic = []

In [4]:
%%time
#Calculating Channels for segemnts
# Count the number of times that NoOfOutlier happens
NoOfOutlier = 0

Stage = [int(i) for i in Stage] #Stage to int

    
#creating empty list for every pair of points
RelativeDistance = [[] for _ in range(len(SegmentNumber))] 
Speed = [[] for _ in range(len(SegmentNumber))]
Acceleration = [[] for _ in range(len(SegmentNumber))]
Jerk = [[] for _ in range(len(SegmentNumber))]
Bearing = [[] for _ in range(len(SegmentNumber))]
BearingRate = [[] for _ in range(len(SegmentNumber))]
Delta_Time = [[] for _ in range(len(SegmentNumber))]
Velocity_Change = [[] for _ in range(len(SegmentNumber))]
User_outlier = []
    
###### Create channels for every Segment (k) 
for k in range(len(SegmentNumber)):
    Data = Data_All_Segment[k] # a list of points in a Segment
    # Temp_RD, Temp_SP are temporary relative distance and speed before checking for their length
    Temp_Speed = []
    Temp_RD = []
    outlier = []
    for i in range(len(Data) - 1):
        A = (Data[i, 0], Data[i, 1])
        B = (Data[i+1, 0], Data[i+1, 1])
        Temp_RD.append(geodesic(A, B).meters)
        Delta_Time[k].append((Data[i + 1, 2] - Data[i, 2]) * 24. * 3600 + 1)  # Add one second to prevent zero time
        S = Temp_RD[i] / Delta_Time[k][i]
        if S > 62.5 or S < 0: # max speed of the fastest rail in the UK
            outlier.append(i)
        Temp_Speed.append(S)
            
        #Calculating Bearing
        y = math.sin(math.radians(Data[i+1, 1]) - math.radians(Data[i, 1])) * math.radians(math.cos(Data[i+1, 0]))
        x = math.radians(math.cos(Data[i, 0])) * math.radians(math.sin(Data[i+1, 0])) - \
        math.radians(math.sin(Data[i, 0])) * math.radians(math.cos(Data[i+1, 0])) \
            * math.radians(math.cos(Data[i+1, 1]) - math.radians(Data[i, 1]))
        # Convert radian from -pi to pi to [0, 360] degree
        b = (math.atan2(y, x) * 180. / math.pi + 360) % 360
        Bearing[k].append(b)

        
    # End of operation of relative distance, speed, and bearing for one Segment
        
    # Now remove all outliers (exceeding max speed) in the current Segment
    Temp_Speed = [i for j, i in enumerate(Temp_Speed) if j not in outlier]        
    if len(Temp_Speed) < 10:
        SegmentNumber[k] = 0
        NoOfOutlier += 1
        continue
    Speed[k] = Temp_Speed
    Speed[k].append(Speed[k][-1])

    # Now remove all outlier Segments, where their speed exceeds the max speed.
    # Then, remove their corresponding points from other channels.
    RelativeDistance[k] = Temp_RD
    RelativeDistance[k] = [i for j, i in enumerate(RelativeDistance[k]) if j not in outlier]
    RelativeDistance[k].append(RelativeDistance[k][-1])
    Bearing[k] = [i for j, i in enumerate(Bearing[k]) if j not in outlier]
    Bearing[k].append(Bearing[k][-1])
    Delta_Time[k] = [i for j, i in enumerate(Delta_Time[k]) if j not in outlier]
    
    SegmentNumber[k] = SegmentNumber[k] - len(outlier) #decrease the number of points in the Segment 

    # Now remove all outlier Segments, where their acceleration exceeds the max acceleration.
    # Then, remove their corresponding points from other channels.
    Temp_ACC = []
    outlier = []
    for i in range(len(Speed[k]) - 1):
        DeltaSpeed = Speed[k][i+1] - Speed[k][i]
        ACC = DeltaSpeed/Delta_Time[k][i]
        if abs(ACC) > 10:
            outlier.append(i)
        Temp_ACC.append(ACC)

    Temp_ACC = [i for j, i in enumerate(Temp_ACC) if j not in outlier]
    if len(Temp_ACC) < 10:
        SegmentNumber[k] = 0
        NoOfOutlier += 1
        continue
    Acceleration[k] = Temp_ACC
    Acceleration[k].append(Acceleration[k][-1])
    Speed[k] = [i for j, i in enumerate(Speed[k]) if j not in outlier]
    RelativeDistance[k] = [i for j, i in enumerate(RelativeDistance[k]) if j not in outlier]
    Bearing[k] = [i for j, i in enumerate(Bearing[k]) if j not in outlier]
    Delta_Time[k] = [i for j, i in enumerate(Delta_Time[k]) if j not in outlier]

    SegmentNumber[k] = SegmentNumber[k] - len(outlier)

    # Now remove all outlier Segments, where their jerk exceeds the max speed.
    # Then, remove their corresponding points from other channels.

    Temp_J = []
    outlier = []
    for i in range(len(Acceleration[k]) - 1):
        Diff = Acceleration[k][i+1] - Acceleration[k][i]
        J = Diff/Delta_Time[k][i]
        Temp_J.append(J)

    Temp_J = [i for j, i in enumerate(Temp_J) if j not in outlier]
    if len(Temp_J) < 10:
        SegmentNumber[k] = 0
        NoOfOutlier += 1
        continue

    Jerk[k] = Temp_J
    Jerk[k].append(Jerk[k][-1])
    Speed[k] = [i for j, i in enumerate(Speed[k]) if j not in outlier]
    Acceleration[k] = [i for j, i in enumerate(Acceleration[k]) if j not in outlier]
    RelativeDistance[k] = [i for j, i in enumerate(RelativeDistance[k]) if j not in outlier]
    Bearing[k] = [i for j, i in enumerate(Bearing[k]) if j not in outlier]
    Delta_Time[k] = [i for j, i in enumerate(Delta_Time[k]) if j not in outlier]

    SegmentNumber[k] = SegmentNumber[k] - len(outlier)
    # End of Jerk outlier detection.

    # Compute Breating Rate from Bearing, and Velocity change from Speed
    for i in range(len(Bearing[k]) - 1):
        Diff = abs(Bearing[k][i+1] - Bearing[k][i])
        BearingRate[k].append(Diff)
    BearingRate[k].append(BearingRate[k][-1])

    for i in range(len(Speed[k]) - 1):
        Diff = abs(Speed[k][i+1] - Speed[k][i])
        if Speed[k][i] != 0:
            Velocity_Change[k].append(Diff/Speed[k][i])
        else:
            Velocity_Change[k].append(1)
    Velocity_Change[k].append(Velocity_Change[k][-1])
        
        
    # Now we apply the smoothing filter on each Segment:
    def savitzky_golay(y, window_size, order, deriv=0, rate=1):
        r"""Smooth (and optionally differentiate) data with a Savitzky-Golay filter.
        The Savitzky-Golay filter removes high frequency noise from data.
        It has the advantage of preserving the original shape and
        features of the signal better than other types of filtering
        approaches, such as moving averages techniques.
        Parameters
        ----------
        y : array_like, shape (N,)
            the values of the time history of the signal.
            window_size : int
            the length of the window. Must be an odd integer number.
        order : int
            the order of the polynomial used in the filtering.
            Must be less then `window_size` - 1.
        deriv: int
            the order of the derivative to compute (default = 0 means only smoothing)
        Returns
        -------
        ys : ndarray, shape (N)
            the smoothed signal (or it's n-th derivative).
        Notes
        -----
        The Savitzky-Golay is a type of low-pass filter, particularly
        suited for smoothing noisy data. The main idea behind this
        approach is to make for each point a least-square fit with a
        polynomial of high order over a odd-sized window centered at
        the point.
        Examples
        --------
        t = np.linspace(-4, 4, 500)
        y = np.exp( -t**2 ) + np.random.normal(0, 0.05, t.shape)
        ysg = savitzky_golay(y, window_size=31, order=4)
        import matplotlib.pyplot as plt
        plt.plot(t, y, label='Noisy signal')
        plt.plot(t, np.exp(-t**2), 'k', lw=1.5, label='Original signal')
        plt.plot(t, ysg, 'r', label='Filtered signal')
        plt.legend()
        plt.show()
        References
        ----------
        .. [1] A. Savitzky, M. J. E. Golay, Smoothing and Differentiation of
        Data by Simplified Least Squares Procedures. Analytical
           Chemistry, 1964, 36 (8), pp 1627-1639.
        .. [2] Numerical Recipes 3rd Edition: The Art of Scientific Computing
        W.H. Press, S.A. Teukolsky, W.T. Vetterling, B.P. Flannery
           Cambridge University Press ISBN-13: 9780521880688
        """
        import numpy as np
        from math import factorial

        try:
            window_size = np.abs(np.int(window_size))
            order = np.abs(np.int(order))
        except ValueError:
            raise ValueError("window_size and order have to be of type int")
        if window_size % 2 != 1 or window_size < 1:
            raise TypeError("window_size size must be a positive odd number")
        if window_size < order + 2:
            raise TypeError("window_size is too small for the polynomials order")
        order_range = range(order + 1)
        half_window = (window_size - 1) // 2
        # precompute coefficients
        b = np.mat([[k ** i for i in order_range] for k in range(-half_window, half_window + 1)])
        m = np.linalg.pinv(b).A[deriv] * rate ** deriv * factorial(deriv)
        # pad the signal at the extremes with
        # values taken from the signal itself
        firstvals = y[0] - np.abs(y[1:half_window + 1][::-1] - y[0])
        lastvals = y[-1] + np.abs(y[-half_window - 1:-1][::-1] - y[-1])
        y = np.concatenate((firstvals, y, lastvals))
        return np.convolve(m[::-1], y, mode='valid')

    # Smoothing process
    #RelativeDistance[k] = savitzky_golay(np.array(RelativeDistance[k]), 9, 3)
    #Speed[k] = savitzky_golay(np.array(Speed[k]), 9, 3)
    #Acceleration[k] = savitzky_golay(np.array(Acceleration[k]), 9, 3)
    #Jerk[k] = savitzky_golay(np.array(Jerk[k]), 9, 3)
    #BearingRate[k] = savitzky_golay(np.array(BearingRate[k]), 9, 3)
        
Total_RelativeDistance.append(RelativeDistance)
Total_Speed.append(Speed)
Total_Acceleration.append(Acceleration)
Total_Jerk.append(Jerk)
Total_BearingRate.append(BearingRate)
Total_Delta_Time.append(Delta_Time)
Total_Velocity_Change.append(Velocity_Change)
Total_Stage.append(Stage)
Total_SegmentNumber.append(SegmentNumber)
Total_Outlier.append(User_outlier)
Total_Segment_InSequence = Total_Segment_InSequence + SegmentNumber

CPU times: user 17min 10s, sys: 868 ms, total: 17min 11s
Wall time: 17min 11s


In [5]:
#saving results
with open('Data/Revised_Unlabeled_60_NoSmoothing_final.pickle', 'wb') as f:
    pickle.dump([Total_RelativeDistance, Total_Speed, Total_Acceleration, Total_Jerk, Total_BearingRate, Total_Stage,
                 Total_SegmentNumber, Total_Segment_InSequence, Total_Delta_Time, Total_Velocity_Change,Total_BusLine,Total_Railway,Total_Traffic], f)